In [1]:
import torch
import math

def bbox_iou(box_wh, anchor_wh):
    """box_wh: (w,h), anchor_wh: (A,2)"""
    box_area = box_wh[0] * box_wh[1]
    anchor_area = anchor_wh[:,0] * anchor_wh[:,1]
    inter = torch.min(box_wh[0], anchor_wh[:,0]) * torch.min(box_wh[1], anchor_wh[:,1])
    union = box_area + anchor_area - inter
    return inter / union

def encode_yolo_v2(labels, anchors, S=13, C=80):
    """
    labels: list of [cls, x, y, w, h] (YOLO txt format, normalized)
    anchors: list of (w,h) in normalized scale (relative to 1)
    return: (S, S, K*(5+C)) tensor
    """
    K = len(anchors)
    label_matrix = torch.zeros((S, S, K*(5+C)))
    anchors = torch.tensor(anchors)

    for cls, x, y, w, h in labels:
        i = int(x * S)  # grid x index
        j = int(y * S)  # grid y index

        # IoU 기준으로 가장 잘 맞는 anchor 선택
        ious = bbox_iou(torch.tensor([w, h]), anchors)
        k = torch.argmax(ious).item()

        # offset 계산
        tx = x * S - i
        ty = y * S - j
        tw = math.log(w / anchors[k,0] + 1e-16)
        th = math.log(h / anchors[k,1] + 1e-16)

        # 위치 채우기
        base = k * (5 + C)
        label_matrix[j, i, base:base+5] = torch.tensor([tx, ty, tw, th, 1.0])
        label_matrix[j, i, base+5+int(cls)] = 1.0  # 클래스 원핫

    return label_matrix


# 예시
label_path = "/home/jinjinjara1022/AutonomousDriving/paper/YOLO_v1/coco128/labels/train2017/000000000025.txt"
with open(label_path) as f:
    labels = [list(map(float, line.strip().split())) for line in f.readlines()]

anchors = [(0.57273,0.677385), (1.87446,2.06253), (3.33843,5.47434), (7.88282,3.52778), (9.77052,9.16828)]
# ↑ COCO에서 YOLOv2가 사용한 anchor (이미지 사이즈 대비 상대 크기)

label_tensor = encode_yolo_v2(labels, anchors, S=13, C=80)

print("라벨 텐서 shape:", label_tensor.shape)  # torch.Size([13,13,425])

# 객체가 들어있는 셀만 출력
nonzero_cells = (label_tensor.sum(dim=-1) > 0).nonzero(as_tuple=False)
for (j, i) in nonzero_cells:
    print(f"Cell ({j},{i}):")
    print(label_tensor[j, i, :])


라벨 텐서 shape: torch.Size([13, 13, 425])
Cell (6,10):
tensor([ 0.0144,  0.3660, -0.5336,  0.0293,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.000

In [2]:
import torch
import torch.nn as nn

# 예시 YOLOv2 헤드 (간단하게만 구현)
class DummyYOLOv2(nn.Module):
    def __init__(self, S=13, B=5, C=80):
        super().__init__()
        self.S, self.B, self.C = S, B, C
        self.out_channels = B * (5 + C)  # per cell
        # 가짜 CNN feature extractor + conv head
        self.conv = nn.Conv2d(512, self.out_channels, kernel_size=1)

    def forward(self, x):
        # x: [batch, 512, S, S]
        return self.conv(x).permute(0, 2, 3, 1)  
        # shape: [batch, S, S, B*(5+C)]

# ---------------------------
# 모델 생성 & 더미 입력
S, B, C = 13, 5, 80
model = DummyYOLOv2(S=S, B=B, C=C)

dummy_x = torch.randn(2, 512, S, S)  # [batch, channels, H, W]
pred = model(dummy_x)

print("모델 출력 shape:", pred.shape)   # torch.Size([2, 13, 13, 425])

# ---------------------------
# 라벨 생성 (YOLOv2 style 라벨)
def dummy_labels(batch_size, S, B, C):
    return torch.zeros((batch_size, S, S, B*(5+C)))

labels = dummy_labels(2, S, B, C)
print("라벨 텐서 shape:", labels.shape)  # torch.Size([2, 13, 13, 425])

# ---------------------------
# 비교
if pred.shape == labels.shape:
    print("✅ 모델 출력과 라벨 shape이 일치합니다!")
else:
    print("❌ 불일치! pred:", pred.shape, "vs label:", labels.shape)


모델 출력 shape: torch.Size([2, 13, 13, 425])
라벨 텐서 shape: torch.Size([2, 13, 13, 425])
✅ 모델 출력과 라벨 shape이 일치합니다!
